<a href="https://colab.research.google.com/github/datasistah/STAR_method/blob/main/StarMethod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q -U git+https://github.com/lvwerra/trl.git git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/accelerate.git git+https://github.com/huggingface/peft.git
!pip install -q -U datasets bitsandbytes einops wandb
!pip install rouge-score


!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers


  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.6 MB/s eta

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


https://vilsonrodrigues.medium.com/run-your-private-llm-falcon-7b-instruct-with-less-than-6gb-of-gpu-using-4-bit-quantization-ff1d4ffbabcc


#### Imports

In [ ]:
#CUDA error message
#!nvcc --version

In [7]:
# !git clone https://github.com/TimDettmers/bitsandbytes.git
# %cd bitsandbytes
# !CUDA_VERSION=118 make cuda11x
# !python setup.py install


In [8]:
#CUDA error message
import os
os.environ["CUDA_VERSION"] = "118"
os.environ["LD_LIBRARY_PATH"] += ":/usr/local/cuda/lib64"


In [9]:
import torch, einops
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft.tuners.lora import LoraLayer

#### Load Quantized Model



In [ ]:
#model_id = "ybelkada/falcon-7b-sharded-bf16"
model_id = "tiiuae/falcon-7b"
compute_dtype = getattr(torch, "float16")

#special config for bitsandbytes that enables 4bit quantization
bnb_config = BitsAndBytesConfig(
              load_in_4bit=True,
              bnb_4bit_quant_type="nf4",
              bnb_4bit_compute_dtype=compute_dtype,
              bnb_4bit_use_double_quant=True,
              )

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
                            model_id,
                            quantization_config=bnb_config,
                            trust_remote_code=True,
                            )


In [12]:
text = '''### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.'''
outputs = model.generate(input_ids=tokenizer(text, return_tensors="pt").input_ids,
                         max_new_tokens=256,
                         top_p=0.9,
                         do_sample=False)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have pu

In [13]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    temperature=0.9,
    repetition_penalty=1.2,
    max_new_tokens=256,
    device_map="auto",
    do_sample=True
)



In [15]:
pipe("a common behaviorial interview question")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


[{'generated_text': "a common behaviorial interview question is to ask why you left your last job.\nIt's not a good question.\nWhy did you leave?\nWhy did you leave?\nBecause she is interviewing for a similar position.\nI'm so frustrated with my job I'm about to snap, and I can't stay any longer.\nI decided I wanted to try something different.\nThey pay isn't horrible, it just doesn't match my skills.\nIt's only temporary.\nIt's only temporary.\nI'm going to be working with the local college to teach their new program.\nI don't like working with students.\nI don't like working with students.\nNot really a good question, but these are possible responses to that question.\nI'll be honest with you...It was boring!\nI had already reached my goals and it was taking the fun out of it."}]

In [27]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"


In [28]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 7.2 MB/s eta 0:00:00


In [35]:
#test PromptTemplate
# from langchain. prompts import PromptTemplate
# multi_prompt= PromptTemplate(input_variables=["adjective", "content"],
#                        template="suggest a good name for the {content} with the genre {adjective}",)

# print(multi_prompt.format(adjective="fictional", content="book"))

suggest a good name for the book with the genre fictional


In [52]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
llm = HuggingFacePipeline(pipeline=pipe)

prompt = PromptTemplate(
      input_variables=['question', 'answer'],
      template="""
    You are a professional career coach designed to help people prepare for behaviorial interview questions.\
    You must determine if they answered using the STAR method for the following question/answer pair.
    question: {question},

    answer: {answer}

    If STAR method detected, provide a summary of each STAR component given in their answer.\
   You must respond in the following format:
    {{
        "Situation": "SITUATION_SUMMARY",
        "Task": "TASK_SUMMARY"
        "Action": "ACTION_SUMMARY"
        "Result": "RESULT_SUMMARY"
    }}
    If they did not use the STAR method, identify the missing components, suggestions and improved answer:\
    You must respond in the following format:
   {{
      "Missing Components": "MISSING_COMPONENTS",
      "Suggestion": "SUGGESTION",
      "Improved Answer": "IMPROVED_ANSWER"

   }}

   Return in json format.
    """
    )

question1 = "Tell me about a time when you received criticism that you thought was unfair."
answer1 = "In a previous role, my manager spoke to me about how long I was speaking to customers for on the telephone. I was trying to build up a positive rapport with our customers to help improve sales. My manager said I was taking too long. Although I felt the criticism was unfair because I was trying to help the company, it was my task to take on board his feedback to improve. I explained to my manager that I was trying to improve customer relations but that I fully understood there was a need to act with speed because other customers were waiting on the line that also needed my assistance. After reflecting on my performance, I made changes as per my manager's instructions and this improved the speed of my call handling rates, meaning I could help and assist more customers."
#test out prompt
print(prompt.format(question=question1, answer = answer1))



    You are a professional career coach designed to help people prepare for behaviorial interview questions.    You must determine if they answered using the STAR method for the following question/answer pair.
    question: Tell me about a time when you received criticism that you thought was unfair., 
    
    answer: In a previous role, my manager spoke to me about how long I was speaking to customers for on the telephone. I was trying to build up a positive rapport with our customers to help improve sales. My manager said I was taking too long. Although I felt the criticism was unfair because I was trying to help the company, it was my task to take on board his feedback to improve. I explained to my manager that I was trying to improve customer relations but that I fully understood there was a need to act with speed because other customers were waiting on the line that also needed my assistance. After reflecting on my performance, I made changes as per my manager's instructions and

In [53]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
response = llm_chain.run(question=question1, answer = answer1)
response

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


'\n         {\n           "error": false,\n           "data": {\n           "message": "The session has been successfully started",\n           "status": "ok"\n           },\n           "success": true\n          }\n\n\n\n\n\n'

#### Prepare Model for Training


*   activate gradient_checkpointing which significally reduces memory
*   print trainable parameters




In [54]:
from peft import prepare_model_for_kbit_training


model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [55]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

#### Create LoRa Config and Appply to Model


*   This is when we add the low ranking adapters/parameters to the model which are the only parameters that will be trained.

*   Increasing the "r" parameter in LoraConfig will add more adapters to the model.



In [56]:
 from peft import LoraConfig, get_peft_model

 peft_config = LoraConfig(
        r=16,
        lora_alpha=32,
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=[
            "query_key_value"
        ]
    )

 model = get_peft_model(model, peft_config)
 print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainable%: 0.13058363808693696


#### Prepare Data

In [57]:
pwd

'/content'

In [60]:
!ls

drive  sample_data  starMethod	STAR_Method_Data.xlsx


In [61]:
import pandas as pd

df = pd.read_excel('/content/STAR_Method_Data.xlsx', sheet_name='Sheet2')
df.head()

,url,Question,Answer,STAR Components
0,https://www.youtube.com/watch?v=2uM7gYuOvr4,Tell me about a time when you received critici...,"In a previous role, my manager spoke to me abo...",Situation: My manager criticized the length of...
1,https://www.youtube.com/watch?v=2uM7gYuOvr4,Tell me about a time when you had to do someth...,I was working in a previous role as part of a ...,Situation: Team member's illness created a gap...
2,https://www.youtube.com/watch?v=2uM7gYuOvr4,Tell me about a time when you worked in a team.,"In a previous role, I was part of a team whose...",Situation: Team responsible for promoting a ne...
3,https://www.youtube.com/watch?v=2uM7gYuOvr4,Tell me about a time when you made a mistake.,"When I started working my last role, I was eag...",Situation: Overcommitted to tasks and missed a...
4,https://www.youtube.com/watch?v=2uM7gYuOvr4,Tell me about a time when you multitasked.,My manager asked for a volunteer to cover the ...,Situation: Volunteered to cover the work of a ...


In [62]:
df = df.sample(frac=1).reset_index(drop=True)
df.tail()

,url,Question,Answer,STAR Components
186,chatgpt,Tell me about a time when you had to work unde...,I had to complete a project within a tight dea...,"Missing Components: Situation, Task, Result <b..."
187,https://www.indeed.com/career-advice/interview...,Tell me about a time when you disagreed with y...,"In my previous role as a Financial Analyst, I ...","Situation: As a Financial Analyst, I disagreed..."
188,https://www.youtube.com/watch?v=tMh0vrQcbmI,Tell me about a time you disagreed with your boss,"In my previous role, my manager was conducting...",Situation: During an annual performance review...
189,https://www.youtube.com/watch?v=2uM7gYuOvr4,Tell me about a time when you failed to meet a...,"In my last role, I was working as part of a te...",Situation: Time-sensitive project with team me...
190,https://novoresume.com/career-blog/interview-s...,Can you give me an example of a time when you ...,"When I was working at Company X, I had to pres...",Situation: Presenting a proposal to the manage...


In [63]:
df["input"] = df[["Question", "Answer"]].apply("-".join, axis=1)
df.head()

,url,Question,Answer,STAR Components,input
0,chatgpt,Tell me about a time when you had to handle a ...,I had to deal with a product launch delay.,"Missing Components: Situation, Task, Action, R...",Tell me about a time when you had to handle a ...
1,chatgpt,Tell me about a time when you had to manage mu...,I had to handle multiple projects simultaneously.,"Missing Components: Situation, Task, Action, R...",Tell me about a time when you had to manage mu...
2,chatgpt,Can you describe a time when you had to deal w...,I had a customer who was not satisfied with ou...,"Missing Components: Situation, Task, Action, R...",Can you describe a time when you had to deal w...
3,https://novoresume.com/career-blog/interview-s...,Can you tell me about a time when you had to w...,When I was working as a content writer for Com...,Situation: A client needed a series of article...,Can you tell me about a time when you had to w...
4,chatgpt,How do you handle stress and pressure?,I take a break and do some deep breathing.,"Missing Components: Situation, Task, Action, R...",How do you handle stress and pressure?-I take ...


In [65]:
df.rename(columns={"STAR Components": 'output'}, inplace=True)
df.head()

,url,Question,Answer,output,input
0,chatgpt,Tell me about a time when you had to handle a ...,I had to deal with a product launch delay.,"Missing Components: Situation, Task, Action, R...",Tell me about a time when you had to handle a ...
1,chatgpt,Tell me about a time when you had to manage mu...,I had to handle multiple projects simultaneously.,"Missing Components: Situation, Task, Action, R...",Tell me about a time when you had to manage mu...
2,chatgpt,Can you describe a time when you had to deal w...,I had a customer who was not satisfied with ou...,"Missing Components: Situation, Task, Action, R...",Can you describe a time when you had to deal w...
3,https://novoresume.com/career-blog/interview-s...,Can you tell me about a time when you had to w...,When I was working as a content writer for Com...,Situation: A client needed a series of article...,Can you tell me about a time when you had to w...
4,chatgpt,How do you handle stress and pressure?,I take a break and do some deep breathing.,"Missing Components: Situation, Task, Action, R...",How do you handle stress and pressure?-I take ...


#### Example Before Fine Tuning

*   using data without training
*   convert text to pytorch tensors



In [119]:
pwd

'/content'

In [67]:
output_dir = "/drive/MyDrive/StarMethod/"

#### Training


*   TRL- HuggingFace "Transformer Reinforcement Learning" library
*   SFTTrainer- "supervised fine-tuning"
*   "paged_adamw_32bit" is powerful optimizer, helps to avoid memory overload



In [120]:
data = df[['input', 'output']]
data.head()

,input,output
0,Tell me about a time when you had to handle a ...,"Missing Components: Situation, Task, Action, R..."
1,Tell me about a time when you had to manage mu...,"Missing Components: Situation, Task, Action, R..."
2,Can you describe a time when you had to deal w...,"Missing Components: Situation, Task, Action, R..."
3,Can you tell me about a time when you had to w...,Situation: A client needed a series of article...
4,How do you handle stress and pressure?-I take ...,"Missing Components: Situation, Task, Action, R..."


In [121]:
len(data)

191

In [86]:
data.to_json('/drive/MyDrive/StarMethod_training_data.jsonl', lines=True, orient='records')


In [122]:
data.head()

,input,output
0,Tell me about a time when you had to handle a ...,"Missing Components: Situation, Task, Action, R..."
1,Tell me about a time when you had to manage mu...,"Missing Components: Situation, Task, Action, R..."
2,Can you describe a time when you had to deal w...,"Missing Components: Situation, Task, Action, R..."
3,Can you tell me about a time when you had to w...,Situation: A client needed a series of article...
4,How do you handle stress and pressure?-I take ...,"Missing Components: Situation, Task, Action, R..."


In [123]:
type(data)

pandas.core.frame.DataFrame

In [124]:
dataset = []
for index, row in data.iterrows():
  input = row['input']
  output = row['output']
  text = f"input:{input} +'' + output:{output}"
  dataset.append(text)

len(dataset)

191

In [125]:
dataset[0]

"input:Tell me about a time when you had to handle a high-pressure situation.-I had to deal with a product launch delay. +'' + output:Missing Components: Situation, Task, Action, Result <br> Suggestion: Provide more details about the product launch, the delay, how you handled the situation, and the outcome. <br> Improved Answer: At my previous job, we were about to launch a new product when we discovered a major issue that caused a delay. Despite the pressure, I coordinated with the team to fix the issue, communicated the delay to our stakeholders, and revised our launch plan. We worked tirelessly to resolve the issue and were able to launch the product successfully a few weeks later."

In [126]:
data['dataset'] = dataset
data.head()


<ipython-input-126-47ef45cf1383>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['dataset'] = dataset


,input,output,dataset
0,Tell me about a time when you had to handle a ...,"Missing Components: Situation, Task, Action, R...",input:Tell me about a time when you had to han...
1,Tell me about a time when you had to manage mu...,"Missing Components: Situation, Task, Action, R...",input:Tell me about a time when you had to man...
2,Can you describe a time when you had to deal w...,"Missing Components: Situation, Task, Action, R...",input:Can you describe a time when you had to ...
3,Can you tell me about a time when you had to w...,Situation: A client needed a series of article...,input:Can you tell me about a time when you ha...
4,How do you handle stress and pressure?-I take ...,"Missing Components: Situation, Task, Action, R...",input:How do you handle stress and pressure?-I...


In [128]:
import pandas as pd
import datasets
from datasets import Dataset, DatasetDict

tds = Dataset.from_pandas(data)


print(tds)

Dataset({
    features: ['input', 'output', 'dataset'],
    num_rows: 191
})


In [ ]:
#import supervised fine-tuning from HF library
#setup the trainer

from trl import SFTTrainer

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=30,
    optim="paged_adamw_32bit",
    save_steps=10,
    logging_steps=1,
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    max_steps=500,
    warmup_ratio=0.03
    )
trainer = SFTTrainer(
    model=model,
    train_dataset=tds,
    peft_config=peft_config,
    dataset_text_field="dataset",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=True,
)
model.config.use_cache = False  # silence the warnings
trainer.train()

#### Save Model


*   local directory
*   HuggingFace



In [131]:
#Save to directory
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)

In [132]:
# import locale
# def getpreferredencoding(do_setlocale = True):
#     return "UTF-8"
# locale.getpreferredencoding = getpreferredencoding

In [135]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cr

In [136]:
!git config --global credential.helper store

In [137]:
#push model to HuggingFace Hub
model_to_save.push_to_hub(repo_id='datasistah/starMethod')

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasistah/starMethod/commit/dda49c775590bca74c7840a2edadd67ba050fda3', commit_message='Upload model', commit_description='', oid='dda49c775590bca74c7840a2edadd67ba050fda3', pr_url=None, pr_revision=None, pr_num=None)

#### Import Model for Inference

In [138]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

config = PeftConfig.from_pretrained("datasistah/starMethod")
model = AutoModelForCausalLM.from_pretrained("tiiuae/falcon-7b", trust_remote_code=True)
model = PeftModel.from_pretrained(model, "datasistah/starMethod")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [140]:
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)

In [141]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
response = llm_chain.run(question=question1, answer = answer1)
response

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


'=> 2+3=?\n   Produce a cleaner output: The manager noticed my customer service skills were lacking. Upon reflection, I identified a way to improve my efficiency. Hence, from now on, I prioritized answering the phone quicker to facilitate better customer care. As a result, my average call handling time improved by 20% over the next month. <br> Improved Answer: In a former role, I learned the importance of prioritizing customers in my work. One day, a customer had an urgent query, but other customers were waiting for assistance. I acknowledged the fact that this was important, but I needed to ensure I provided excellent service to all customers. Reflecting on my performance, I determined a change in my approach would ensure I could prioritize customers waiting. This resulted in further improvements to my call handling times.'

In [144]:
print(response)

=> 2+3=?
   Produce a cleaner output: The manager noticed my customer service skills were lacking. Upon reflection, I identified a way to improve my efficiency. Hence, from now on, I prioritized answering the phone quicker to facilitate better customer care. As a result, my average call handling time improved by 20% over the next month. <br> Improved Answer: In a former role, I learned the importance of prioritizing customers in my work. One day, a customer had an urgent query, but other customers were waiting for assistance. I acknowledged the fact that this was important, but I needed to ensure I provided excellent service to all customers. Reflecting on my performance, I determined a change in my approach would ensure I could prioritize customers waiting. This resulted in further improvements to my call handling times.


In [147]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
response = llm_chain.run(question="Tell me about a time when you received criticism that you thought was unfair.", answer = "In a previous role, my manager spoke to me about how long I was speaking to customers for on the telephone. I was trying to build up a positive rapport with our customers to help improve sales. My manager said I was taking too long. Although I felt the criticism was unfair because I was trying to help the company, it was my task to take on board his feedback to improve. I explained to my manager that I was trying to improve customer relations but that I fully understood there was a need to act with speed because other customers were waiting on the line that also needed my assistance. After reflecting on my performance, I made changes as per my manager's instructions and this improved the speed of my call handling rates, meaning I could help and assist more customers.")
response

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


"> Lorem Ipsum is used as a filler text for a form template. <br> Good Answer <br> Your answer should be at least one paragraph long. Where's the problem? What actions did you take? And how did you manage to solve the issue? Lorem Ipsum"

In [148]:
question3 = "What is your greatest achievement?"
answer3 = "When I was a manager at Cafe X, we would always get a lot more customers than usual during the summer. It would almost always be packed, and sometimes, there were not enough servers to handle the work. So, I had to come up with a system so that we would not get overwhelmed during those 3+ months. To make sure we were prepared, I realized we needed a better schedule. For the next summer, I had a very strict shift schedule ready, and a reward system in case someone couldn’t show up for their shift. If someone called in sick, whoever filled in for them would be rewarded with extra pay. And if there was absolutely no one else available, I would personally fill in for them instead. Overall, the summer went pretty smoothly with minimal accidents. We didn’t get ANY negative reviews on TripAdvisor for the summer (whereas for the summer before that, we had several for slow service)."

llm_chain = LLMChain(prompt=prompt, llm=llm)
response = llm_chain.run(question=question3, answer = answer3)
response


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


" Paste Code Snippet if you could improve performance by implementing new technologies or procedures. Describe how you handled the transition from the old technology to the new one. Did you set aside time for training employees? Was there any resistance? How did you deal with it?\n  +-+'' => Output:In my previous role as a Software Engineer, I was responsible for upgrading our company's outdated coding software. My task was to research different code generators and select the best option to suit our needs. After extensive screening, I identified a coding generator which was used by similar companies within our industry. The switchover took place smoothly and we experienced no disruption to our workflow. As an unexpected benefit, we were able to streamline our coding process, reducing the time spent writing code significantly. -> Solution A: Provide details about the situation, the actions taken, and the outcome. Commute: Start--> Task: Manage a team remotely.---------------------------